In [1]:
import numpy as np
import pandas as pd

# Parameters
fs = 100  # Sampling frequency (Hz)
duration = 600  # Duration of signal in seconds (10 minutes) 
t = np.arange(0, duration, 1/fs)  # Time vector

# Function to generate synthetic respiration signal
def generate_respiration_signal():
    # Base respiration signal (normal breathing)
    respiration_signal = np.sin(2 * np.pi * 0.2 * t)  # 0.2 Hz (12 breaths per minute)
    
    # Add noise to make the signal more realistic
    noise = 0.1 * np.random.normal(size=t.shape)
    respiration_signal += noise

    # Simulate apnea events (80% reduction in airflow)
    apnea_events = np.random.choice([0, 1], size=t.shape, p=[0.95, 0.05])  # 5% chance of apnea
    apnea_durations = np.random.randint(5, 15, size=np.sum(apnea_events))  # Varying duration of apnea events
    for i, event in enumerate(np.where(apnea_events == 1)[0]):
        respiration_signal[event:event+apnea_durations[i]] *= 0.2  # 80% reduction

    # Simulate hypopnea events (30% reduction in airflow)
    hypopnea_events = np.random.choice([0, 1], size=t.shape, p=[0.9, 0.1])  # 10% chance of hypopnea
    hypopnea_durations = np.random.randint(10, 30, size=np.sum(hypopnea_events))  # Varying duration of hypopnea events
    for i, event in enumerate(np.where(hypopnea_events == 1)[0]):
        respiration_signal[event:event+hypopnea_durations[i]] *= 0.7  # 30% reduction
    
    return respiration_signal, apnea_events, hypopnea_events

# Generate multiple recordings
num_recordings = 10
data_list = []

for i in range(num_recordings):
    respiration_signal, apnea_events, hypopnea_events = generate_respiration_signal()
    data = pd.DataFrame({
        'time': t,
        'nasal_airflow': respiration_signal,
        'apnea_event': apnea_events,
        'hypopnea_event': hypopnea_events,
        'event': np.where(apnea_events == 1, 'apnea', np.where(hypopnea_events == 1, 'hypopnea', 'normal'))
    })
    data['recording'] = i
    data_list.append(data)

# Combine all recordings into a single DataFrame
combined_data = pd.concat(data_list, ignore_index=True)

# Save to CSV
combined_data.to_csv('enhanced_respiration_data.csv', index=False)